# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pathlib import Path

In [ ]:
from PIL import Image
# importing required libraries of opencv
import cv2

# importing library for plotting
from matplotlib import pyplot as plt

In [ ]:
from scipy.optimize import curve_fit
from scipy.interpolate import make_interp_spline

In [ ]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

# from IPython.display import display, Image

# Run Model

In [ ]:
model = YOLO("/kaggle/input/basketball-models-trajectory/best_yolo8_v2_ep_50.pt")

In [ ]:
# %cd {HOME}
# !yolo task=detect mode=predict model=/kaggle/input/basketball-models-trajectory/best_yolo8_v2_ep_50.pt conf=0.5 source=/kaggle/input/basketball-small-videos-1-60fps/60.MP4 save=True

# Find Trajectory

In [ ]:
def draw_bbox(img_arr, res_list):
    backboard_cord = []
    ball_cord      = []
    hoop_cord      = []
    width_ratio    = []
    ball_width     = 0
    hoop_width     = 0
    frame_cat      = set()
    ball_conf      = 0
    hoop_conf      = 0
    
    for val in res_list:
        bbox = val[:4]
        conf = round(val[4], 2)
        cat  = val[5]
        
        if conf<0.5:
            continue
        
        pt1   = (int(bbox[0]), int(bbox[1]))
        pt2   = (int(bbox[2]), int(bbox[3]))
        centx = int((int(bbox[0]) + int(bbox[2]))/2)
        centy = int((int(bbox[1]) + int(bbox[3]))/2)    
        
        if cat == 0:
            label = 'backboard'
            backboard_cord.append([centx, centy])
        elif cat == 1:
            label = 'ball'
            if conf>ball_conf:
                ball_cord = []
                ball_cord.append([centx, centy])
                ball_width = abs(int(bbox[0]) - int(bbox[2]))
            frame_cat.add("ball")
        elif cat ==2:
            label = 'hoop'
            if conf>hoop_conf:
                hoop_cord = []
                hoop_cord.append([centx, centy])
                hoop_width = abs(int(bbox[0]) - int(bbox[2]))
            frame_cat.add('hoop')
        
        # print(pt1, pt2)
        cv2.rectangle(img_arr,pt1, pt2,(0,255,0),2)
        cv2.putText(img_arr, f'{label} {conf}', 
                    (pt1[0], pt1[1]-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.9, (0,255,0), 2)
        cv2.circle(img_arr, (centx,centy), radius=4, color=(0, 0, 255), thickness=-1)
        
    if ball_width == 0 or hoop_width==0:
        width_ratio = None
    else:
        width_ratio = hoop_width/ball_width
        
    if len(frame_cat)!=2:
        return False, img_arr, None, None, None, None
        
    return True, img_arr, backboard_cord, ball_cord, hoop_cord, width_ratio

In [ ]:
def analysis_projectile_data(hoop_list, ball_list):
    
    def polarity_change_no(list1):
        if list1[0] == 0:
            prev = -1
        else:
            prev = list1[0] / abs(list1[0])
        ans = 0

        for elem in list1:
            if elem == 0:
                sign = -1
            else:
                sign = elem / abs(elem)

            if sign == -prev:
                ans = ans + 1
                prev = sign

        return ans
    
    def avg_pt(list1, n):
        return [sum(list1[i:i+n])//n 
                for i in range(0,len(list1),n) 
                if i+n<len(list1)]
    
    x1 = [val[0] for val in hoop_list]
    y1 = [-val[1] for val in hoop_list]
    
    x2 = [val[0] for val in ball_list]
    y2 = [-val[1] for val in ball_list]
    
    xr = [b-a for a, b in zip(x1,x2)]
    yr = [b-a for a, b in zip(y1,y2)]
    
    xr_clean = []
    yr_clean = []
    
    for index, (a, b) in enumerate(zip(xr, yr)):
        if abs(a) < 200 and abs(b)<200:
            xr_clean.append(a)
            yr_clean.append(b)
    
    xr_clean_avg_pt = avg_pt(xr_clean, 7)
    yr_clean_avg_pt = avg_pt(yr_clean, 7)
    
    print(f"With clean data size: {len(xr_clean)}")
    print(f"With clean avg data size: {len(xr_clean_avg_pt)}")
    if len(xr_clean_avg_pt)<3:
        return False
    slope = np.diff(yr_clean_avg_pt)/np.diff(xr_clean_avg_pt)

    ball_direction = all(i < j for i, j in zip(xr_clean_avg_pt,xr_clean_avg_pt[1:]))
    
    ball_final_status = []
    
    if ball_direction:
        ball_final_status.append('right')
        print("right direction")
    else:
        ball_final_status.append('left')
        print("left direction")
    
    direction_change = polarity_change_no(slope)
    if direction_change>0:
        ball_final_status.append('hit')
        print("bounce back")
    else:
        ball_final_status.append('pass')
        print("passthrough")
        
    return ball_final_status

In [ ]:
def draw_text(image, text, row):
    font = cv2.FONT_HERSHEY_SIMPLEX
    # org
    org = (50, 100*row)

    # fontScale
    fontScale = 1

    # Blue color in BGR
    color = (0, 0, 255)

    # Line thickness of 2 px
    thickness = 2

    # Using cv2.putText() method
    image = cv2.putText(image, text, org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    return image

In [ ]:
def find_trajectory(path):
    cap = cv2.VideoCapture(path)
    
    # Check if camera opened successfully
    if (cap.isOpened()== False):
        print("Error opening video file") 
    
    # We need to set resolutions.
    # so, convert them from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    size = (frame_width, frame_height)
    
    # video info
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    length = total_frame_count/fps
    print(f'Total Frames: {total_frame_count}')
    print(f'Frame Rate: {fps}')
    print(f'Video Length: {length}')
    
    # to save the video
    os.makedirs('/kaggle/working/outputs/', exist_ok=True)
    video_name = Path(path).stem
    output_video_path = f'/kaggle/working/outputs/{video_name}.mp4'
    writer= cv2.VideoWriter(output_video_path , 
                            cv2.VideoWriter_fourcc(*'DIVX'), 
                            int(fps), 
                            size)
    
    count = 0
#     pbar = tqdm(total = total_frame_count)
    
    back_list = []
    ball_list = []
    hoop_list = []
    
    width_ratio_list = []
    till_first_analysis = False
    start_point_status  = True
    one_run             = False
    start_index         = 1
    end_index           = 0
    # Read until video is completed
    while(cap.isOpened()):
            
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            # pbar.update(count)
            count = count + 1
            if (count%1000==0): print(count)
            
            result = model(frame)
            result_list = result[0].boxes.boxes.tolist()
            status, frame, back, ball, hoop, ratio = draw_bbox(frame, result_list)
#             print(ratio)
            if status:
                if ratio>1.3:
                    if start_point_status:
                        print(f"Start Analysis: {start_index}")
                        start_point_status = False
                    if ratio>1.5:
                        frame = draw_text(frame, 
                                          f"Start Analysis: {start_index}",
                                          1)
                        till_first_analysis = True
                        one_run             = True
                        if status:
                            back_list = back_list + back
                            ball_list = ball_list + ball
                            hoop_list = hoop_list + hoop
                            width_ratio_list.append(ratio)
                else:
                    if one_run:
                        ball_direction = analysis_projectile_data(hoop_list, ball_list)
                        one_run = False
                        start_point_status = True
                        if ball_direction==False:
                            print("No Data")
                            start_index = start_index + 1
                            end_index = end_index +1
                            continue
                        start_index = start_index + 1
                        end_index = end_index +1
                        print(f"End Analysis: {end_index}")
                    
                    back_list = []
                    ball_list = []
                    hoop_list = []
                    width_ratio_list = []    
            
            if not one_run and till_first_analysis:
                frame = draw_text(frame, 
                                  f"End Analysis {end_index}: {ball_direction}",
                                  2)
            writer.write(frame)
        # Break the loop
        else:
            break
            
    if till_first_analysis:
        ball_direction = analysis_projectile_data(hoop_list, ball_list)
        print(ball_direction)
        
    
    cap.release()
    writer.release()
#     cv2.destroyAllWindows()
    return back_list, ball_list, hoop_list, width_ratio_list

In [ ]:
%%time
# back_list, ball_list, hoop_list, w_ratio = find_trajectory('/kaggle/input/basketball-small-videos-1-60fps/60.MP4')
back_list, ball_list, hoop_list, w_ratio = find_trajectory('/kaggle/input/basketball-small-videos-1-60fps/traj_2.MP4')

In [ ]:
def plot_projectile(lines):
    names = ['backboard', 'ball', 'hoop']
    for i in range(1, 3, 1):
        for index, (name, line) in enumerate(zip(names, lines)):
            # create data
            x = [val[0] for val in line][:i*40]
            y = [-val[1] for val in line][:i*40]
    #         print(len(x))

            # plot lines
            plt.plot(x, y, 'bo', color=plt.cm.RdYlBu(np.linspace(0,1, len(lines))[index]), label = f"{name}")
    #         plt.plot(x, y,  'o', label = f"line {index}")

        plt.legend()
        plt.show()

In [ ]:
w_ratio

In [ ]:
plot_projectile([back_list, ball_list, hoop_list])

In [ ]:
def plot_poly_curve(arr1, arr2):
    def polarity_change_no(list1):
        # Variable Initialization
        if list1[0] == 0:
            prev = -1
        else:
            prev = list1[0] / abs(list1[0])
        ans = 0

        # Using Iteration
        for elem in list1:
            if elem == 0:
                sign = -1
            else:
                sign = elem / abs(elem)

            if sign == -prev:
                ans = ans + 1
                prev = sign

        # Printing answer
        return ans
    
    def avg_pt(list1, n):
        return [sum(list1[i:i+n])//n 
                for i in range(0,len(list1),n) 
                if i+n<len(list1)]
    
    x1 = [val[0] for val in arr1]
    y1 = [-val[1] for val in arr1]
    
    x2 = [val[0] for val in arr2]
    y2 = [-val[1] for val in arr2]
    
    xr = [b-a for a, b in zip(x1,x2)]
    yr = [b-a for a, b in zip(y1,y2)]
    
#     # calculate polynomial
#     z = np.polyfit(xr, yr, 3)
#     f = np.poly1d(z)

#     # calculate new x's and y's
#     x_new = np.linspace(min(xr), max(xr), 50)
#     y_new = f(x_new)
    xr_clean = []
    yr_clean = []
    
    for index, (a, b) in enumerate(zip(xr, yr)):
        if abs(a) < 200 and abs(b)<200:
            xr_clean.append(a)
            yr_clean.append(b)
    
    xr_clean_avg_pt = avg_pt(xr_clean, 7)
    yr_clean_avg_pt = avg_pt(yr_clean, 7)
    print(len(xr_clean), len(xr_clean_avg_pt))

    m = np.diff(yr_clean_avg_pt)/np.diff(xr_clean_avg_pt)

    plt.plot(xr_clean, yr_clean)
#     plt.xlim([xr[0]-1, xr[-1] + 1 ])
    plt.show()
    
    ball_direction = all(i < j for i, j in zip(xr_clean_avg_pt,xr_clean_avg_pt[1:]))
    if ball_direction:
        print("right direction")
    else:
        print("left direction")
        
    plt.plot(xr_clean_avg_pt, yr_clean_avg_pt, "o")
#     plt.xlim([xr[0]-1, xr[-1] + 1 ])
    plt.show()
    
    direction_change = polarity_change_no(m)
    if direction_change>0:
        print("bounce back")
    else:
        print("passthrough")
        
    plt.plot(m, "o")
    plt.axhline(y=0, color = 'y', label = 'thres line')
    plt.show()

In [ ]:
zip([1,2], [3])

In [ ]:
plot_poly_curve(hoop_list, ball_list)

In [ ]:
def get_direction(xr, yr):
    vector_xr = xr[-1]-xr[0]
    yr_min = min(yr)
    if yr_min>0:
        if vector_xr<0:
            return 'left'

In [ ]:
def plot_relative(arr1, arr2, w_ration):
    x1 = [val[0] for val in arr1]
    y1 = [-val[1] for val in arr1]
    
    x2 = [val[0] for val in arr2]
    y2 = [-val[1] for val in arr2]
    
    xr = [b-a for a, b in zip(x1,x2)]
    yr = [b-a for a, b in zip(y1,y2)]   
    
    index_above_thres_list = []
    for index in range(len(xr)):
        if w_ratio[index]!=None and w_ratio[index]>1.5:
            index_above_thres_list.append(index)
            
    min_index = index_above_thres_list[0]
    max_index = index_above_thres_list[-1]
    
    direction = get_direction(xr[min_index:max_index], 
                              yr[min_index:max_index])
    print(direction)
    if direction == 'left':
        plt.text(0,60, f"{direction}", bbox=dict(facecolor='red', alpha=0.5))
#         plt.arrow(150,700, 100,1,width=0.5, head_width=0.5, head_length=0.5)
#         plt.annotate(s='', xy=(150,-100), xytext=(150,100), arrowprops=dict(arrowstyle='<->'))
    
    plt.axvline(x = xr[min_index], color = 'y', label = 'thres line')
    plt.axvline(x = xr[max_index], color = 'y', label = 'thres line')

    # plot relative position
    plt.plot(0, 25, 'ks', markerfacecolor='none', ms=30, markeredgecolor='red')
    # plot lines
    plt.plot(xr, yr, 'bo', label = f"relative distance")
#         plt.plot(x, y,  'o', label = f"line {index}")

    plt.legend()
    plt.show()

In [ ]:
# [i for i in w_ratio if i>2]

In [ ]:
plot_relative(hoop_list, back_list, w_ratio)

In [ ]:
plot_relative(hoop_list, ball_list, w_ratio)

In [ ]:
# w_ratio

In [ ]:
len(w_ratio), len(back_list), len(ball_list), len(hoop_list)

In [ ]:
plot_relative(back_list, ball_list, w_ratio)

# Raugh

In [ ]:
img = Image.open('/kaggle/input/extra-data/1.PNG').convert('RGB') 
img_arr = np.array(img) 

In [ ]:
gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, threshold1=30, threshold2=100)

In [ ]:
Image.fromarray(edges)

In [ ]:
pil_image = Image.fromarray(img_arr)

In [ ]:
pil_image

In [ ]:
def draw_bbox(img_arr, res_list):
    backboard_cord = []
    ball_cord      = []
    hoop_cord      = []
        
    for val in res_list:
        bbox = val[:4]
        conf = round(val[4], 2)
        cat  = val[5]
        
        pt1   = (int(bbox[0]), int(bbox[1]))
        pt2   = (int(bbox[2]), int(bbox[3]))
        centx = (int(bbox[0]) + int(bbox[2]))/2
        centy = (int(bbox[1]) + int(bbox[3]))/2    
        
        if cat == 0:
            label = 'backboard'
            backboard_cord.append([centx, centy])
        elif cat == 1:
            label = 'ball'
            ball_cord.append([centx, centy])
        elif cat ==2:
            label = 'hoop'
            hoop_cord.append([centx, centy])
        
        # print(pt1, pt2)
        cv2.rectangle(img_arr,pt1, pt2,(0,255,0),2)
        cv2.putText(img_arr, f'{label} {conf}', 
                    (pt1[0], pt1[1]-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.9, (0,255,0), 2)
        
    return img_arr, backboard_cord, ball_cord, hoop_cord

In [ ]:
img = Image.open('/kaggle/input/extra-data/1.PNG').convert('RGB') 
img_arr = np.array(img) 

In [ ]:
result      = model(pil_image)
result_list = result[0].boxes.boxes.tolist()
img_arr, back, ball, hoop = draw_bbox(img_arr, result_list)

In [ ]:
[back, ball, hoop+['1', '2']]

In [ ]:
def plot_projectile(lines):
    for index, line in enumerate(lines):
        # create data
        x = [val[0] for val in line]
        y = [val[1] for val in line]

        # plot lines
        plt.plot(x, y, 'bo', color=plt.cm.RdYlBu(np.linspace(0,1, len(lines))[index]), label = f"line {index}")
#         plt.plot(x, y,  'o', label = f"line {index}")

    plt.legend()
    plt.show()
    
plot_projectile([back, ball, hoop])

In [ ]:
Image.fromarray(img_arr)